In [1]:
import mlcvs


/home/lbonati@iit.local/software/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import torch 
import pytorch_lightning as pl
from typing import Any

from mlcvs.core import FeedForward, Normalization
from mlcvs.utils.data import TensorDataModule
from torch.utils.data import TensorDataset

from mlcvs.core.utils.decorators import decorate_methods,call_submodules_hooks,allowed_hooks

@decorate_methods(call_submodules_hooks,methods=allowed_hooks)
class SimpleCV(pl.LightningModule):
    """Example of collective variable obtained with a regression task.
    Initalize the architecture as a torch.nn.sequential model and the 
    loss function (MSE) in the training_step function. """
    
    def __init__(self, options : dict[str, Any] = None , modules : list = None, **kwargs):
        """Initialize the CV either from a dictionary or with a list of pl.LightningModules for greater flexibility. 

        If instead a list of modules is given they will supersede the options dict.

        Parameters
        ----------
        options : dict[str, Any], optional
            The allowed keys are :
            - layers  :  list of nodes per layers (e.g. [n_input, n_hidden, n_output]).
            - activation : type of activation function (see FeedForward class) 
            - normIn  : how to normalize inputs (std,minmax,False/None), default std
        modules : list of pl.LightningModules, alternative
            list of modules, by default None
        """
        super().__init__(**kwargs)

        if ( modules is not None ):
            self.blocks = torch.nn.Sequential(*modules)
        else:
            # parse layers
            try:
                layers = options['layers']
            except KeyError:
                raise KeyError('the key layers (e.g. [n_input, n_hidden, n_output]) is missing in the options dict')
            n_in = layers[0] 
            # parse activation
            activation = options['activation'] if 'activation' in options else 'relu'
            # create NN
            nn = FeedForward(layers, activation=activation)
            # norm inputs
            modules = []
            if ( 'normIn' in options ):
                normIn = options['normIn']
                if (normIn != False) and (normIn is not None):
                    if normIn == True:
                        normIn = 'std'
                    mode = normIn
                    modules.append(Normalization(n_in,mode=normIn))
            modules.append(nn)
                       
            self.blocks = torch.nn.Sequential(*modules)

        self.example_input_array = torch.ones(modules[0].n_in)

        # parameters
        self.n_in = modules[0].n_in
        self.n_out = modules[-1].n_out
        self.lr = 1e-3

    def forward(self, x: torch.tensor) -> (torch.tensor):
        y = self.blocks(x)
        return y

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def loss_function(self, input, target):
        # MSE LOSS
        loss = (input-target).square().mean()
        return loss

    def training_step(self, train_batch, batch_idx):
        x, labels = train_batch
        y = self(x)
        loss = self.loss_function(y,labels)
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, labels = val_batch
        y = self(x)
        loss = self.loss_function(y,labels)
        self.log('val_loss', loss, on_epoch=True)

def test_simplecv():
    n_in, n_out = 2,1 
    layers = [n_in, 5, 10, n_out]

    # initialize via dictionary
    options= { 'layers' : layers ,
               'activation' : 'relu',
               'normIn' : True } 
    model = SimpleCV( options )
    print('----------')

    # initialize via list of modules
    modules = [ Normalization(n_in),
                FeedForward(layers, activation='tanh') ]
    model = SimpleCV( modules=modules )
    print(model)

    # create dataset
    X = torch.randn((100,2))
    y = X.square().sum(1)
    dataset = TensorDataset(X,y)
    datamodule = TensorDataModule(dataset,lengths=[0.75,0.2,0.05], batch_size=25)
    # train model
    trainer = pl.Trainer(accelerator='cpu',max_epochs=2,logger=None)
    trainer.fit( model, datamodule )
    # trace model
    traced_model = model.to_torchscript(file_path="model_trace.pt", method='trace', example_inputs=X[0])
    model.eval()
    assert torch.allclose(model(X),traced_model(X))

if __name__ == "__main__":
    test_simplecv() 

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type       | Params | In sizes | Out sizes
-------------------------------------------------------------
0 | blocks | Sequential | 86     | [2]      | [1]      
-------------------------------------------------------------
86        Trainable params
0         Non-trainable params
86        Total params
0.000     Total estimated model params size (MB)


----------
SimpleCV(
  (blocks): Sequential(
    (0): Normalization()
    (1): FeedForward(
      (nn): Sequential(
        (0): Linear(in_features=2, out_features=5, bias=True)
        (1): Tanh()
        (2): Linear(in_features=5, out_features=10, bias=True)
        (3): Tanh()
        (4): Linear(in_features=10, out_features=1, bias=True)
      )
    )
  )
)
Epoch 1: 100%|██████████| 4/4 [00:00<00:00, 274.14it/s, loss=5.34]          

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 4/4 [00:00<00:00, 237.47it/s, loss=5.34]
